In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import cdsapi

# c = cdsapi.Client()

# years = list(range(1950, 2023))

# for year in years:
#     c.retrieve(
#         'reanalysis-era5-pressure-levels',
#         {
#             'product_type': 'reanalysis',
#             'variable': [
#                 'divergence', 'fraction_of_cloud_cover', 'geopotential',
#                 'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
#                 'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
#                 'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
#                 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
#                 'vorticity',
#             ],
#             'pressure_level': '500',
#             'year': '2022',
#             'month': [
#                 '01', '02', '03',
#                 '04', '05', '06',
#                 '07', '08', '09',
#                 '10', '11', '12',
#             ],
#             'day': [
#                 '01', '02', '03',
#                 '04', '05', '06',
#                 '07', '08', '09',
#                 '10', '11', '12',
#                 '13', '14', '15',
#                 '16', '17', '18',
#                 '19', '20', '21',
#                 '22', '23', '24',
#                 '25', '26', '27',
#                 '28', '29', '30',
#                 '31',
#             ],
#             'time': '00:00',
#             'area': [
#                 38.28899868843215, -122.02635927086914, 36.169668221548285,
#                 -119.58014102395923,
#             ],
#             'format': 'netcdf',
#         },
#         'era5land_' + str(year) + '.nc')
#     print('era5land_' + str(year) + '.nc' + ' downloaded.')



In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [4]:
! pip install geemap wxee

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [5]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio
import numpy as np
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping


In [6]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/private-key.json"
)

ee.Initialize(credentials)

In [11]:
# Define your points of interest as a list of tuples (longitude, latitude)
points = [
    (-119.58014102395923,36.4587205291358),
    (-120.20100395913578,36.169668221548285),
    (-120.10236864383359,36.94660861179753),
    (-120.20264490832344,36.177667848838176),
    (-120.20999796596058,36.83900158517578),
    (-121.11800025437275,38.28899868843215),
    (-121.49932919916024,38.099152461081694),
    (-121.65310129898836,38.10872169194239),
    (-121.64669802987558,38.11590084463843),
    (-121.53500200758782,38.108998156207704),
    (-121.64330019487288,38.10469958279218),
    (-121.75470191649107,38.03689895127454),
    (-122.02635927086914,38.2005568781381)
    ]




# Create a list of ee.Geometry.Point objects
point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in points]
# Define the ERA5 variables you are interested in
variables = [
    "dewpoint_temperature_2m",
    "temperature_2m",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    "lake_total_layer_temperature",
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall_sum",
    "snowmelt_sum",
    "temperature_of_snow_layer",
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4",
    "forecast_albedo",
    "surface_latent_heat_flux_sum",
    "surface_net_solar_radiation_sum",
    "surface_net_thermal_radiation_sum",
    "surface_sensible_heat_flux_sum",
    "surface_solar_radiation_downwards_sum",
    "surface_thermal_radiation_downwards_sum",
    "evaporation_from_bare_soil_sum",
    "evaporation_from_open_water_surfaces_excluding_oceans_sum",
    "evaporation_from_the_top_of_canopy_sum",
    "evaporation_from_vegetation_transpiration_sum",
    "potential_evaporation_sum",
    "runoff_sum",
    "snow_evaporation_sum",
    "sub_surface_runoff_sum",
    "surface_runoff_sum",
    "total_evaporation_sum",
    "u_component_of_wind_10m",
    "v_component_of_wind_10m",
    "surface_pressure",
    "total_precipitation_sum",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation",
    "dewpoint_temperature_2m_min",
    "dewpoint_temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_max",
    "skin_temperature_min",
    "skin_temperature_max",
    "soil_temperature_level_1_min",
    "soil_temperature_level_1_max",
    "soil_temperature_level_2_min",
    "soil_temperature_level_2_max",
    "soil_temperature_level_3_min",
    "soil_temperature_level_3_max",
    "soil_temperature_level_4_min",
    "soil_temperature_level_4_max",
    "lake_bottom_temperature_min",
    "lake_bottom_temperature_max",
    "lake_ice_depth_min",
    "lake_ice_depth_max",
    "lake_ice_temperature_min",
    "lake_ice_temperature_max",
    "lake_mix_layer_depth_min",
    "lake_mix_layer_depth_max",
    "lake_mix_layer_temperature_min",
    "lake_mix_layer_temperature_max",
    "lake_shape_factor_min",
    "lake_shape_factor_max",
    "lake_total_layer_temperature_min",
    "lake_total_layer_temperature_max",
    "snow_albedo_min",
    "snow_albedo_max",
    "snow_cover_min",
    "snow_cover_max",
    "snow_density_min",
    "snow_density_max",
    "snow_depth_min",
    "snow_depth_max",
    "snow_depth_water_equivalent_min",
    "snow_depth_water_equivalent_max",
    "snowfall_min",
    "snowfall_max",
    "snowmelt_min",
    "snowmelt_max",
    "temperature_of_snow_layer_min",
    "temperature_of_snow_layer_max",
    "skin_reservoir_content_min",
    "skin_reservoir_content_max",
    "volumetric_soil_water_layer_1_min",
    "volumetric_soil_water_layer_1_max",
    "volumetric_soil_water_layer_2_min",
    "volumetric_soil_water_layer_2_max",
    "volumetric_soil_water_layer_3_min",
    "volumetric_soil_water_layer_3_max",
    "volumetric_soil_water_layer_4_min",
    "volumetric_soil_water_layer_4_max",
    "forecast_albedo_min",
    "forecast_albedo_max",
    "surface_latent_heat_flux_min",
    "surface_latent_heat_flux_max",
    "surface_net_solar_radiation_min",
    "surface_net_solar_radiation_max",
    "surface_net_thermal_radiation_min",
    "surface_net_thermal_radiation_max",
    "surface_sensible_heat_flux_min",
    "surface_sensible_heat_flux_max",
    "surface_solar_radiation_downwards_min",
    "surface_solar_radiation_downwards_max",
    "surface_thermal_radiation_downwards_min",
    "surface_thermal_radiation_downwards_max",
    "evaporation_from_bare_soil_min",
    "evaporation_from_bare_soil_max",
    "evaporation_from_open_water_surfaces_excluding_oceans_min",
    "evaporation_from_open_water_surfaces_excluding_oceans_max",
    "evaporation_from_the_top_of_canopy_min",
    "evaporation_from_the_top_of_canopy_max",
    "evaporation_from_vegetation_transpiration_min",
    "evaporation_from_vegetation_transpiration_max",
    "potential_evaporation_min",
    "potential_evaporation_max",
    "runoff_min",
    "runoff_max",
    "snow_evaporation_min",
    "snow_evaporation_max",
    "sub_surface_runoff_min",
    "sub_surface_runoff_max",
    "surface_runoff_min",
    "surface_runoff_max",
    "total_evaporation_min",
    "total_evaporation_max",
    "u_component_of_wind_10m_min",
    "u_component_of_wind_10m_max",
    "v_component_of_wind_10m_min",
    "v_component_of_wind_10m_max",
    "surface_pressure_min",
    "surface_pressure_max",
    "total_precipitation_min",
    "total_precipitation_max",
    "leaf_area_index_high_vegetation_min",
    "leaf_area_index_high_vegetation_max",
    "leaf_area_index_low_vegetation_min",
    "leaf_area_index_low_vegetation_max"
]

In [24]:
# Define the time range for the data
start_date = '2008-12'
end_date = '2021-01'

# Load the ERA5 dataset
era5 = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').filterDate(start_date, end_date)

# Initialize an empty dictionary to store data for each point
data = {}

for i, point in enumerate(point_geometries):
    point_data = {}
    for var in variables:
        # Select the variable from the ERA5 dataset
        era5_var = era5.select(var)

        # Reduce the image collection to extract data at the point
        point_reduction = era5_var.map(lambda img: img.sample(point, scale=11132).first()).toList(era5_var.size())

        # Convert the reduced data to a list of dictionaries
        point_data[var] = point_reduction.getInfo()

    data[f'Point_{i}'] = point_data

# Now `data` holds the ERA5 data for each point and variable


In [25]:
import csv

# Define the CSV file name

csv_filename = '/content/drive/MyDrive/EC_Tower/result/era5_monthly_data.csv'

# Write data to CSV
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Point', 'Variable', 'Date', 'Value']
    writer.writerow(header)

    for point_key, point_data in data.items():
        for var, records in point_data.items():
            for record in records:
                date = record['id'].split('_')[-1]  # Extract date from ID
                value = record['properties'][var]
                writer.writerow([point_key, var, date, value])

print(f"Data has been saved to {csv_filename}")


Data has been saved to /content/drive/MyDrive/EC_Tower/result/era5_monthly_data.csv
